In [1]:
import json
import pandas as pd
import os

In [6]:
# directory = os.getcwd()
directory = "C:/Users/ajarvinen/OneDrive - KPMG/Desktop/polar-user-data-export_e8293a83-b2e8-47b9-9545-ea2de41e79bb/cycling/sekalainen"

# Iterate through all files in the folder
for filename in os.listdir(directory):
    if filename.endswith(".json") and "training-session" in filename: 
        transformJson(filename)
        continue
    else:
        continue

In [7]:
# directory = os.getcwd()
directory = "C:/Users/ajarvinen/OneDrive - KPMG/Desktop/polar-user-data-export_e8293a83-b2e8-47b9-9545-ea2de41e79bb/cycling/sekalainen/"

# Test with single filename
filename = 'training-session-2020-08-01-4964709011-5a45d8c4-bae0-44f1-9104-49717c990a16.json'

# Extract the time string
time = filename[17:38]

# Read json
f = open(directory+filename)
data = json.load(f)

# Check the type of the training
# Currently supported: CYCLING
sport = data['exercises'][0]['sport']

# CYCLING:
if sport == "CYCLING":
    df_route = pd.json_normalize(data['exercises'][0]['samples']['recordedRoute'])
    df_heartRate = pd.json_normalize(data['exercises'][0]['samples']['heartRate'])
    df_speed = pd.json_normalize(data['exercises'][0]['samples']['speed'])
    df_distance = pd.json_normalize(data['exercises'][0]['samples']['distance'])

    # Create merged dataframe
    merged = pd.merge(df_heartRate,df_speed,on="dateTime")
    merged = pd.merge(merged,df_distance,on="dateTime")

    # Create key columns based on seconds (rounded values)
    s = df_route['dateTime'].astype(str)
    df_route['key'] = s.str[0:19]

    s = merged['dateTime'].astype(str)
    merged['key'] = s.str[0:19]

    # Merge all data - use outer join to preserve all data
    merged = pd.merge(merged,df_route,on="key",how="outer")

    # drop, rename, reorganize columns
    merged = merged.drop(['dateTime_y'], axis=1)
    merged = merged.rename(columns={"value_x": "heartRate", "value_y": "speed", "value" : "distance", "dateTime_x" : "dateTime"})
    merged = merged[['key','dateTime','longitude','latitude','altitude','heartRate','speed','distance']]

    # Write csv
    name = directory+"/"+sport+"_"+time+".csv"
    merged.to_csv(name,index=False)

In [2]:
def transformJson(filename):

    # Extract the time string
    time = filename[17:38]

    # Read json
    f = open(filename)
    data = json.load(f)

    # Check the type of the training
    # Currently supported: CYCLING
    sport = data['exercises'][0]['sport']

    # CYCLING:
    if sport == "CYCLING":
        df_route = pd.json_normalize(data['exercises'][0]['samples']['recordedRoute'])
        df_heartRate = pd.json_normalize(data['exercises'][0]['samples']['heartRate'])
        df_speed = pd.json_normalize(data['exercises'][0]['samples']['speed'])
        df_distance = pd.json_normalize(data['exercises'][0]['samples']['distance'])

        # Create merged dataframe
        merged = pd.merge(df_heartRate,df_speed,on="dateTime")
        merged = pd.merge(merged,df_distance,on="dateTime")

        # Create key columns based on seconds (rounded values)
        s = df_route['dateTime'].astype(str)
        df_route['key'] = s.str[0:19]

        s = merged['dateTime'].astype(str)
        merged['key'] = s.str[0:19]

        # Merge all data - use outer join to preserve all data
        merged = pd.merge(merged,df_route,on="key",how="outer")

        # drop, rename, reorganize columns
        merged = merged.drop(['dateTime_y'], axis=1)
        merged = merged.rename(columns={"value_x": "heartRate", "value_y": "speed", "value" : "distance", "dateTime_x" : "dateTime"})
        merged = merged[['key','dateTime','longitude','latitude','altitude','heartRate','speed','distance']]

        # Write csv
        name = directory+"/"+sport+"_"+time+".csv"
        merged.to_csv(name,index=False)

In [29]:
# Print dataframe
merged

,key,dateTime,longitude,latitude,altitude,heartRate,speed,distance
0,2020-07-16T07:57:42,2020-07-16T07:57:42.650,24.813097,60.228458,45.0,107,0.00,0.000000
1,2020-07-16T07:57:43,2020-07-16T07:57:43.650,24.813073,60.228450,45.0,106,0.00,0.000000
2,2020-07-16T07:57:44,2020-07-16T07:57:44.650,24.813058,60.228445,46.0,103,0.00,0.000000
3,2020-07-16T07:57:45,2020-07-16T07:57:45.650,24.813038,60.228433,46.0,102,0.00,0.000000
4,2020-07-16T07:57:46,2020-07-16T07:57:46.650,24.813028,60.228430,46.0,102,0.00,0.000000
...,...,...,...,...,...,...,...,...
2065,2020-07-16T08:33:00,2020-07-16T08:33:00.650,24.877782,60.281548,27.0,105,6.13,9422.200195
2066,2020-07-16T08:33:01,2020-07-16T08:33:01.650,24.877778,60.281583,27.0,106,6.25,9422.200195
2067,2020-07-16T08:33:02,2020-07-16T08:33:02.650,24.877787,60.281580,26.0,106,6.27,9422.200195
2068,2020-07-16T08:33:03,2020-07-16T08:33:03.650,24.877737,60.281578,27.0,107,6.22,9422.900391
